In [34]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
import seaborn as sns
import pandas as pd
import cv2 as cv

In [35]:
file_path = "../IRCCyN_IVC_DIBR_Videos_Scores.xls"

df_base= pd.read_excel(file_path, skiprows=[0])
nb = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32]
df_cleaned = df_base.drop(columns=nb)
df_videos = df_cleaned.drop(columns=['Unnamed: 33', 'std'])
df_videos = df_videos.rename(columns={'Unnamed: 0': 'Video'})
df_videos.head()

,Video,MOS,CI
0,Fehn_c_Book_arrival_bh_8_to_10,2.21875,0.270256
1,Fehn_c_Book_arrival_bh_10_to_8,2.15625,0.301427
2,Fehn_c_Book_arrival_sh_8_to_9,3.40625,0.298299
3,Fehn_c_Book_arrival_sh_10_to_9,3.56250,0.258958
4,Fehn_i_Book_arrival_bh_8_to_10,2.03125,0.280474


In [37]:
#divide the video column into two columns: one for the aldorithm (first part of the title) and one for the video ID (Book_arrival, Lovebird1 or Newspaper)
df_videos["Algo"] = np.nan
df_videos["Video_ID"] = np.nan
for index, row in df_videos.iterrows():
    video_name = row["Video"]
    #check whether there is Book_arrival, Lovebird1 or Newspaper in the video name
    if "Book_arrival" in video_name or "book_arrival" in video_name:
        df_videos.at[index, "Video_ID"] = "Book_arrival"
    elif "Lovebird1" in video_name or "lovebird1" in video_name:
        df_videos.at[index, "Video_ID"] = "Lovebird1"
    elif "Newspaper" in video_name or "newspaper" in video_name:
        df_videos.at[index, "Video_ID"] = "Newspaper"

    if "Fehn_c" in video_name:
        df_videos.at[index, "Algo"] = "Fehn_c"
    elif "Fehn_i" in video_name:
        df_videos.at[index, "Algo"] = "Fehn_i"
    elif "Holes" in video_name:
        df_videos.at[index, "Algo"] = "Holes"
    elif "ICIP_TMM" in video_name:
        df_videos.at[index, "Algo"] = "ICIP_TMM"
    elif "ICME" in video_name:
        df_videos.at[index, "Algo"] = "ICME"
    elif "MPEG" in video_name:
        df_videos.at[index, "Algo"] = "MPEG"
    elif "Muller" in video_name:
        df_videos.at[index, "Algo"] = "Muller"
    else : 
        df_videos.at[index, "Algo"] = "Original"
print(df_videos)    

                              Video      MOS        CI      Algo      Video_ID
0    Fehn_c_Book_arrival_bh_8_to_10  2.21875  0.270256    Fehn_c  Book_arrival
1    Fehn_c_Book_arrival_bh_10_to_8  2.15625  0.301427    Fehn_c  Book_arrival
2     Fehn_c_Book_arrival_sh_8_to_9  3.40625  0.298299    Fehn_c  Book_arrival
3    Fehn_c_Book_arrival_sh_10_to_9  3.56250  0.258958    Fehn_c  Book_arrival
4    Fehn_i_Book_arrival_bh_8_to_10  2.03125  0.280474    Fehn_i  Book_arrival
..                              ...      ...       ...       ...           ...
97                 newspaper_cam_05  4.59375  0.190946  Original     Newspaper
98                 newspaper_cam_06  4.75000  0.150031  Original     Newspaper
99            newspaper_cam_04_qp26  4.34375  0.268515  Original     Newspaper
100           newspaper_cam_04_qp34  3.09375  0.234986  Original     Newspaper
101           newspaper_cam_04_qp44  1.21875  0.143235  Original     Newspaper

[102 rows x 5 columns]


/tmp/ipykernel_69235/1144674060.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Book_arrival' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_videos.at[index, "Video_ID"] = "Book_arrival"
/tmp/ipykernel_69235/1144674060.py:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Fehn_c' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_videos.at[index, "Algo"] = "Fehn_c"
